In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
import numpy as np
import glob
import os
from IPython.display import Image,display

import pickle
import time

Using TensorFlow backend.


In [2]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')

In [3]:
cat['category_label'].value_counts()

41    72158
18    36887
3     24557
32    19666
17    15429
33    14773
6     13311
16    13123
11    10467
19    10078
2      7495
48     7408
26     7076
42     6153
30     5013
29     4416
10     4048
34     3048
44     2294
39     2120
24     1669
35     1106
15      791
12      748
9       716
13      676
27      594
22      527
23      486
36      386
5       330
7       324
4       309
1       160
47      150
20      146
43      126
14       97
21       77
46       70
37       54
25       49
28       45
31       32
8        17
40       17
Name: category_label, dtype: int64

In [7]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input
def crop_resize_DF(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

In [45]:
N = 2000
start = time.time()
subset = np.random.randint(bbox.shape[0],size=N)
proc_img = []
for i in subset:
    proc_img.append(np.squeeze(preprocess_DF(bbox.iloc[i])))
proc_img = np.array(proc_img)
print('{:.3f} s'.format(time.time()-start))

14.145 s


In [46]:
from multiprocessing import Pool
p = Pool()
def proc(i):
    return preprocess_DF(bbox.iloc[i])

In [47]:
start = time.time()
r = np.squeeze(p.map(proc,subset))
print('{:.3f} s'.format(time.time()-start))

8.784 s


In [35]:
np.squeeze(r).shape

(10, 224, 224, 3)

In [19]:
bbox.iloc[subset]

,image_name,x_1,y_1,x_2,y_2
89273,img/Faux_Fur-Lined_Parka/img_00000063.jpg,33,31,201,295
140986,img/Mid-Rise_-_Distressed_Skinny_Jeans/img_000...,111,2,211,283
63125,img/Champagne_Slub_Knit_Top/img_00000008.jpg,100,72,208,266
47865,img/Tasseled_Tribal_Print_Cardigan/img_0000004...,77,34,214,263
8545,img/Abstract_Paint_Splatter_Blouse/img_0000006...,1,16,200,268
20171,img/Floral_Crochet_Cutout_Blouse/img_00000008.jpg,43,1,264,300
157497,img/Shimmering_High-Waist_Leggings/img_0000002...,95,1,205,300
285986,img/Collarless_A-Line_Boucl&eacute;_Coat/img_0...,35,36,162,208
279957,img/Crisscross-Back_Skater_Dress/img_00000054.jpg,66,13,244,279
273067,img/Ornate_Lace_Romper/img_00000007.jpg,1,48,162,261
